<a href="https://colab.research.google.com/github/K4cp3rski/ML_FUW/blob/dev/Cybi%C5%84ski_02_zadanie_domowe_dropped_entities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Praca domowa I, zadanie II





## Treść

### Wstęp fabularny

Wyobraź sobie, że jesteś pracownikiem w firmie sprzedającej kompleksową usługę tworzenia wizerunków medialnych. Oddział, w którym pracujesz obsługuje ważnego klienta działającego w branży gier i usług cyfrowych.

Twoim zadaniem jest przygotować model uczenia maszynowego, który określać będzie nastawienie emocjonalne postów z Twittera. Zespół odpowiadający za zbieranie danych właśnie dostarczył zestaw danych dla Ciebie.

Do tej pory klasyfikowaniem nastrojów z twittów zajmował się zespół ekspertów. Rozwiązanie takie jest bardzo wolne i drogie, a dokładność ekspertów wynosi tylko 95%. Dlatego zarząd firmy zlecił wdrożenie modelu uczenia maszynowego.

Twój model stanowić będzie jedynie część większego produktu oferowanego przez Twoją firmę. Wyniki Twojego modelu będą bezpośrednio wykorzystywane przez następny zespół, którego zadaniem jest przygotować kolejny model uczenia maszynowego przewidujący reakcje opinii publicznej na posty klienta.

Prace zespołu, który korzystać będzie z Twojego modelu są już bardzo zaawansowane, dlatego nie może on pozwolić sobie na żadne dodatkowe zmiany w swoim projekcie. Absolutnie konieczne jest, aby Twój model przyporządkowywał posty do jednej z trzech klas 'Positive', 'Negative', 'Neutral' lub analogicznych. Posty nie na temat powinny być klasyfikowane jako 'Neutral'.

Notebook z Twoim projektem będzie oglądał Twój szef, więc koniecznie zadbaj, żeby znalazły się w nim najważniejsze przemyślenia, a rysunki były ładne.

Powodzenia 🦾

### Polecenia

1. Wstępna obróbka danych:

 - załaduj zbiór treningowy i testowy,
 - usuń wiersze o brakujących elementach,
 - w kolumnie `sentiment` zamień wartości `'Irrelevant'` na `'Neutral'`.

1. Wykonaj wizualizacje danych:

 - histogram tematów twittów (`entity`),
 - histogram nastawień (`sentiment`),
 - najczęściej padających słów w treści twittów (`content`).

1. Przygotuj dane:

 - przygotuj zbiór cech poprzez wektoryzacje kolumny `content`, 
 - przygotuj etykiety poprzez zakodowanie tekstowych wartości w kolumnie `sentiment` do postaci liczbowej.

  Następnie wytrenuj naiwny model bayesowski. Sprawdź działanie modelu na kilku własnoręcznie napisanych wiadomościach. 

1. Wytrenuj modele:
 - naiwny bayesowski,
 - liniowy SVM,
 - regresji logistycznej,
 - drzewo decyzyjne.

  Sprawdź model na danych treningowych (walidacja krzyżowa) i testowych, następnie wybierz najlepszy model. Uzasadnij swój wybór.
  
1. Zespół ekspertów ręcznie klasyfikuje dane z dokładnością 95%. Porównaj z nimi swój model i napisz jakie są przewagi Twojego modelu.

### Zbiór danych

Zbiór danych został przygotowany na podstawie zbioru [Twitter Sentiment Analysis](https://www.kaggle.com/jp797498e/twitter-entity-sentiment-analysis) i składa się z dwóch plików:
-  `twitter_training.csv` - zbiór treningowy,
- `twitter_validation.csv` - zbiór testowy.

Archiwum z plikami można pobrać z [dysku google](https://drive.google.com/file/d/1sw2vA87fmAI5V5Xl9k-PCSdN5XwydhOB/view?usp=sharing) lub odkomentowując poniższe linie:

In [91]:
# ! pip install gdown
# ! gdown https://drive.google.com/uc?id=1sw2vA87fmAI5V5Xl9k-PCSdN5XwydhOB
# ! unzip twitter.zip

In [92]:
# !pip install wordcloud

## Rozwiązanie

In [93]:
import pandas as pd
import numpy as np
import seaborn as sns

import sklearn
print('Zainstalowana wersja scikit-learn: {}.'.format(sklearn.__version__))

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,10) # aby wykresy w Colabie były większe

import numpy as np
from scipy import diag, interp
from itertools import cycle

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn import metrics

Zainstalowana wersja scikit-learn: 1.0.1.


In [94]:
train_data = pd.read_csv('twitter_training.csv')
test_data = pd.read_csv('twitter_validation.csv')

In [95]:
# # Wizualizacja danych

# f = sns.countplot(x='sentiment', data=train_data)
# h = sns.catplot(y='entity', hue='sentiment', data=train_data, kind='count', height=20, aspect=1)
# g = sns.catplot(x='sentiment', col='entity' , col_wrap=4, kind='count', data=train_data, height=4.5, aspect=1.2)

# (g.set_axis_labels("", "Tweet count")
# .set_xticklabels(["Positive", "Neutral", "Negative", 'Irrelevant'])
# .set_titles("{col_name}")
# .despine(left=True))  

Jak widzimy globalnie (Obrazek 1) klasy są w miare zbalansowane, liczebność nie różni się więcej jak 2-krotnie. W związku z tym wydaje się, że nie ma potrzeby sztucznego wyrównywania ich liczebności i można przejść do dalszego etapu preprocessingu danych

In [96]:
# Przydzielamy klasom integerowe labelki

# Neutral = Class 0
train_data.loc[train_data['sentiment'] == 'Positive', 'sentiment'] = 0
# Positive = Class 1
train_data.loc[train_data['sentiment'] == 'Neutral', 'sentiment'] = 1
# Negative = Class 2
train_data.loc[train_data['sentiment'] == 'Negative', 'sentiment'] = 2
# Irrelevant = Class 1
train_data.loc[train_data['sentiment'] == 'Irrelevant', 'sentiment'] = 1

# A teraz to samo dla zbiotu testowego

# Neutral = Class 0
test_data.loc[test_data['sentiment'] == 'Positive', 'sentiment'] = 0
# Positive = Class 1
test_data.loc[test_data['sentiment'] == 'Neutral', 'sentiment'] = 1
# Negative = Class 2
test_data.loc[test_data['sentiment'] == 'Negative', 'sentiment'] = 2
# Irrelevant = Class 1
test_data.loc[test_data['sentiment'] == 'Irrelevant', 'sentiment'] = 0

In [97]:
# print(train_data['sentiment'].unique())
# print(test_data['sentiment'].unique())

In [98]:
train_data = train_data.drop(columns=['id', 'entity'])
test_data = test_data.drop(columns=['id', 'entity'])

train_data = train_data.rename(columns={'sentiment':'Class'})
test_data = test_data.rename(columns={'sentiment':'Class'})

# Pozbywamy się wierszy z niepełnymi informacjani (NaN)
train_data = train_data.dropna(axis='rows', how='all', thresh=int(train_data.shape[1]))
test_data = test_data.dropna(axis='rows', how='all', thresh=int(test_data.shape[1]))

In [99]:
# # Zobaczmy sobie najczęściej występujące słowa w klasach słów pozytywnych, neutralnych, negatywnych i irrelevant

# import wordcloud
# from wordcloud import WordCloud
# import matplotlib.pyplot as plt

# # Neutralne tweety
# neutral_list = list(map(str, train_data [train_data['Class'] == 0]['content']))

# neutral_words = " ".join(neutral_list)
# neutral_plot = WordCloud(width = 512, height = 512).generate(neutral_words)

# # Pozytywne tweety
# positive_list = list(map(str, train_data [train_data['Class'] == 1]['content']))

# positive_words = " ".join(positive_list)
# positive_plot = WordCloud(width = 512, height = 512).generate(positive_words)

# # Negatywne tweety
# negative_list = list(map(str, train_data [train_data['Class'] == 2]['content']))

# negative_words = " ".join(negative_list)
# negative_plot = WordCloud(width = 512, height = 512).generate(negative_words)

# plt.figure(figsize=(10,8))

In [100]:
# images = [neutral_plot, positive_plot, negative_plot]
# image_names = ['neutral', 'positive', 'negative']


# plt.figure(figsize=(20,5))
# plt.suptitle('Most frequent words in tweets class-wise',y = 0.95, x = 0.4, weight='heavy', size='xx-large')
# columns = 4
# for i, image in enumerate(images):
#     plt.subplot(len(images) / columns + 1, columns, i + 1)
#     plt.axis('off')
#     plt.title(image_names[i])
#     plt.imshow(image)

In [101]:
from sklearn.model_selection import train_test_split

X_train = train_data['content']
y_train = np.asarray(train_data['Class']).reshape(-1, 1).ravel()
y_train = y_train.astype('int')

X_test = test_data['content']
y_test = np.asarray(test_data['Class']).reshape(-1, 1).ravel()
y_test = y_test.astype('int')

Teraz jak już mamy zwizualizowane dane, to przekodujmy je na język zrozumiały przez maszynę, tj. wektory cech.

 W tym celu użyjemy funkcji *CountVectorizer*

In [102]:
from sklearn.feature_extraction.text import CountVectorizer

# Dopasowanie i wektoryzowanie dla danych treningowych

vectorizer = CountVectorizer() # stwórz instancje obiektu CountVectorizer dla kodowania tekstu tweetów
X_train = vectorizer.fit_transform(X_train) # naucz vectorizer słownika i przetransformuj dane uczące (kolumna treści)

# Wektoryzowanie i przetransformowanie danych testowych korzystając ze słownika stworzonego na bazie danych treningowych

X_test = vectorizer.transform(X_test) # naucz vectorizer słownika i przetransformuj dane uczące (kolumna treści)

In [103]:
print("Dane treningowe: n_samples: %d, n_features: %d" % X_train.shape)

Dane treningowe: n_samples: 73996, n_features: 31062


Odwrotne mapowanie cech na słowa

In [104]:
# Dla kolumny z tekstem tweetów

feature_names = vectorizer.get_feature_names_out()
feature_names = np.asarray(feature_names)

---
### Tworzymy instancję klasyfikatora MultinomialNB

In [105]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

---
### Sprawdźmy na zbiorze testowym

Robimy predykcję dla X_test

In [106]:
y_pred = clf.predict(X_test) # obliczamy predykcję dla tekstów ze zbioru testowego

In [107]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

accur = accuracy_score(y_test, y_pred) # dokładność
print("Dokładność: %0.3f" % accur)
print("Classification report:") # wypisz raport klasyfikacji 

print(classification_report(y_test, y_pred))

print("Macierz błędów") # wypisz macierz (confusion matrix)
print(confusion_matrix(y_test, y_pred))

print("\nPrecyzja wyliczona built-in method = {}".format(clf.score(X_test, y_test)))

Dokładność: 0.731
Classification report:
              precision    recall  f1-score   support

           0       0.86      0.60      0.71       449
           1       0.59      0.80      0.68       285
           2       0.77      0.89      0.82       266

    accuracy                           0.73      1000
   macro avg       0.74      0.76      0.74      1000
weighted avg       0.76      0.73      0.73      1000

Macierz błędów
[[268 144  37]
 [ 24 227  34]
 [ 19  11 236]]

Precyzja wyliczona built-in method = 0.731


### Możemy nasz klasyfikator ulepszyć, w tym celu trzeba poddać dane stemmingowi

Jednak jako, że jedna z naszych kolumn zawiera tylko nazwy gier/tematów o których jest pisane, to stemmingowi chcemy poddać jedynie kolumnę z treścią tweetów, bo to tam szukanie słów o wspólnych korzeniach znaczeniowych będzie mieć znaczenie

In [112]:
# Importy niezbędnych rzeczy

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [109]:
stemmer = PorterStemmer()
analyzer = CountVectorizer().build_analyzer()

words = stopwords.words("english")
train_data['cleaned'] = train_data['content'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())
test_data['cleaned'] = test_data['content'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())

def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

In [113]:
# Czyścimy X_train i X_test

X_train = train_data['cleaned']
X_test = test_data['cleaned']


# Dopasowanie i wektoryzowanie dla danych treningowych wraz ze stemmingiem

vectorizer = TfidfVectorizer(min_df= 3, stop_words="english", sublinear_tf=True, norm='l2', ngram_range=(1, 2)) # stwórz instancje obiektu CountVectorizer dla kodowania tekstu tweetów
X_train = vectorizer.fit_transform(X_train) # naucz vectorizer słownika i przetransformuj dane uczące (kolumna treści)

# Wektoryzowanie i przetransformowanie danych testowych korzystając ze słownika stworzonego na bazie danych treningowych

X_test = vectorizer.transform(X_test) # naucz vectorizer słownika i przetransformuj dane uczące (kolumna treści)



# <--- !! ---> #



# # Odwrotne mapowanie tweetów na słowa

# Dla kolumny z tekstem tweetów

feature_names = vectorizer.get_feature_names_out()
feature_names = np.asarray(feature_names)



# <--- !! ---> #



# # # Klasyfikator

clf = MultinomialNB()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
accur = accuracy_score(y_test, y_pred) # dokładność
print("Dokładność: %0.3f" % accur)
print("Classification report:") # wypisz raport klasyfikacji 
print(classification_report(y_test, y_pred))

print("Macierz błędów") # wypisz macierz (confusion matrix)
print(confusion_matrix(y_test, y_pred))

Dokładność: 0.798
Classification report:
              precision    recall  f1-score   support

           0       0.98      0.59      0.73       449
           1       0.60      0.98      0.74       285
           2       0.96      0.97      0.96       266

    accuracy                           0.80      1000
   macro avg       0.85      0.84      0.81      1000
weighted avg       0.87      0.80      0.80      1000

Macierz błędów
[[263 178   8]
 [  4 278   3]
 [  2   7 257]]


#Jest to dokładność o 15 p.p. niższa niż zostawiając kolumnę 'entities'. 
## Ta droga nie będzie kontynuowana